In [1]:
!pip3 install datasets
!pip3 install tqdm
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [2]:
import numpy as np
import pandas as pd

from datasets import Dataset
from tqdm import tqdm

In [3]:
import torch

# set gpu if available
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'


import os
import multiprocessing

num_cores = multiprocessing.cpu_count()

In [4]:
def read_data(dataset='train'):
  def read_from_hf():
    import huggingface_hub
    huggingface_hub.login()

    from datasets import load_dataset
    dataset = load_dataset("aeirya/irhw3", use_auth_token=True)

    import pandas as pd
    text = dataset['train']['text']

    with open('buffer.txt', 'w') as file:
      file.write('\n'.join(text))

    df = pd.read_csv('buffer.txt', delimiter='\t')

    return df

  def read_from_drive():
    from google.colab import drive
    try:
      drive.mount('/content/drive')
    except:
      pass

    dir = '/content/drive/My Drive/Assignment/InformationRetrieval/hw3/dataset'
    file = f'{dir}/{dataset}_dataset.txt'

    return pd.read_csv(file, delimiter='\t')

  return read_from_drive()

df = read_data('train')
df.head()

Mounted at /content/drive


,user_id,item_id,rate,review_text
0,A2YKWYC3WQJX5J,B00106AC06,1,I usually love the Motions conditioners and ma...
1,A2LXC5ZHHP0WXP,B00AE07BMQ,1,Axe messy look styling gum is a product that w...
2,A3HLTHHLPKLRQA,B00AIQOKDY,1,I have always found liquid soap to be as much ...
3,A6N1DC5AMPLSK,B000F6RFX4,1,I've tried plenty of products that claim to he...
4,ALNFHVS3SC4FV,B0020122ZS,1,Suave Kids is one of my favorite brands of sha...


In [5]:
# from data_reader import read_interaction_matrix as interactions, read_df

# R = interactions()
# df = read_df('train')
# df.head()

In [6]:
users = list(set(df['user_id']))
items = list(set(df['item_id']))

n_users = len(users)
n_items = len(items)

user2idx = { x:i for i,x in enumerate(users) }
item2idx = { x:i for i,x in enumerate(items) }

In [7]:
import torch

def init_model_tokenizer(model_name):
  from transformers import AutoTokenizer, AutoModel

  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModel.from_pretrained(model_name)

  model.eval()
  model.to(device)

  return model, tokenizer


# model_name = 'bert-base-uncased'
# model_name = 'distilbert-base-uncased'
model_name = 'distilroberta-base'

model, tokenizer = init_model_tokenizer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

In [8]:
df = df.dropna()

# checking the distribution of review token lengths
tok_no_pad = tokenizer(df['review_text'].tolist(), padding=False, truncation=True)
n = [len(e.ids) for e in tok_no_pad[:]]

pd.Series(n).describe(percentiles=[0.5, 0.6, 0.7, 0.8, 0.9])

count    23037.000000
mean       162.225420
std        111.393901
min          4.000000
50%        133.000000
60%        157.000000
70%        189.000000
80%        235.000000
90%        320.000000
max        512.000000
dtype: float64

In [11]:
def tok(sentences, max_length=200):
    return tokenizer(
        sentences,
        add_special_tokens=True,
        return_tensors="pt",
        return_attention_mask=True,
        padding=True,
        truncation=True,
        max_length=max_length,
        # pad_to_max_length=True
    )


def bert(texts, tokenized=True):
    tok_input = tok(texts) if not tokenized else texts

    # note: if not tokenized then must move input to gpu

    with torch.no_grad():
        output = model(**tok_input).last_hidden_state

    return output


def sent2vec(output, alpha=0.2):
    '''
    aggregate tokens in every sentence together

    alpha: the effect of max pooling

    @param output: (n_sentences x n_tokens x n_hidden_size)
    '''

    max_ = output.max(axis=1).values
    mean = output.mean(axis=1)
    return alpha * max_ + (1-alpha) * mean


def init_ds(df):
  # note: df should not containnan
  ds = Dataset.from_pandas(df)\
    .map(tok, batched=True, batch_size=80000, input_columns='review_text', remove_columns=['review_text'])

  ds.set_format('torch')

  print("dataset ready")

  return ds


def bert_encode_items(df):
  df = df.dropna().sort_values('item_id').reset_index()[['item_id', 'review_text']]

  input_ds = init_ds(df)

  # encoding matrix of items
  enc = torch.zeros((len(items), 768))

  # reviews for each item
  groups = df.groupby(by='item_id', dropna=True)

  for item, group in tqdm(groups):

      input = input_ds[group.index]
      input = {k:input[k].to(device) for k in ['input_ids', 'token_type_ids', 'attention_mask'] if k in input}

      out = bert(input)

      # average of all sentences
      v = sent2vec(out).mean(axis=0)

      enc[item2idx[item], :] = v

  return enc


def pca(M):
  '''
  reduces dimension of nxm matrix M to nx64
  '''

  from sklearn.decomposition import PCA

  R = PCA(64).fit_transform(M)

  return R

In [26]:
R = np.ones((len(users), len(items)))
Q = np.ones((len(users), 64))
P = np.ones((64, len(items)))

In [27]:
E = bert_encode_items(df)

# part A:
Pa = pca(E)
Ra = Q @ Pa.T

# part B:
# Mistake in the problem: P is the corresponding matrix for items
Pb = pca(np.hstack((E, P.T)))
Rb = Q @ Pb.T